In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.spatial
import pandas as pd
import sklearn.decomposition
import matplotlib.pyplot as plt
# import keras
from sklearn import preprocessing,linear_model
from sklearn.metrics import pairwise_distances,mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from utils.readProfiles import *
from utils.pred_models import *
from yellowbrick.model_selection import FeatureImportances

sns.set_style("whitegrid")
# from utils import networksEvol, tsne, readProfiles
import umap

##### create single blue colormap to change defualt colors in yellowbrick
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
blue_cmap = cm.get_cmap('Blues', 200)
single_blue_cmap=ListedColormap(blue_cmap(np.linspace(0.7, 0.9, 20)))

## Read treatment level profiles

In [7]:
procProf_dir='/home/ubuntu/datasetsbucket/Rosetta-GE-CP/'
# results_dir='./results/'

ge_features_under_test=['CISD1']
dataset='LUAD';

################################################
pertColName='PERT'

mergProf_treatLevel,cp_features,l1k_features = \
read_paired_treatment_level_profiles(procProf_dir,dataset,'normalized','highRepOverlap',1)


l1k=mergProf_treatLevel[[pertColName]+l1k_features]
cp=mergProf_treatLevel[[pertColName]+cp_features]

scaler_ge = preprocessing.StandardScaler()
scaler_cp = preprocessing.StandardScaler()
l1k_scaled=l1k.copy()
l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
cp_scaled=cp.copy()
cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))


if 1:
    cp=cp_scaled.copy()
    l1k=l1k_scaled.copy()

/home/ubuntu/workspace_rosetta/workspace/software/2021_Haghighi_NeurIPS_Dataset_submitted/utils/readProfiles.py:42: DtypeWarning: Columns (1023,1028) have mixed types. Specify dtype option on import or set low_memory=False.
  l1k_data_repLevel=pd.read_csv(dataDir+'/L1000/replicate_level_l1k.csv.gz')


LUAD: Replicate Level Shapes (nSamples x nFeatures): cp:  6144 , 1569 ,  l1k:  4232 , 978
l1k n of rep:  8.0
cp n of rep:  8.0
CP: from  593  to  364
l1k: from  529  to  275
CP and l1k high rep overlap:  197
Treatment Level Shapes (nSamples x nFeatures+metadata): (198, 1570) (198, 979) Merged Profiles Shape: (198, 2548)


## For a selected landmark gene, here how we sample images:
- We fit a linear model to treatment level profiles
- We filter the sample to the one that have confident predictions (sample that have l1 norm (true, pred) less than 0.3)
- We samples select highest and lowest prediction values in this filtered set
- We read replicate level profiles
- We randomly select one replicate (pert replicates are at the same well of 8 different plates)
- We select site in the middle of the well (s5 out of 9 available sites)

In [5]:
dataset='LUAD'
profileType='normalized_variable_selected'
per_plate_normalized_flag=1
[cp_data_repLevel,cp_features], [l1k_data_repLevel,l1k_features]=\
read_replicate_level_profiles(procProf_dir,dataset,profileType,per_plate_normalized_flag)


#####################################################
dict_imag_lookup={}
# luad_genes_to_inspect_ims=['PRKAG2','BNIP3L','NIPSNAP1','MYO10']
# luad_genes_to_inspect_ims=['ETFB','CISD1','SPP1', 'CDK4', 'S100A4','TSC22D3','MRPL19','TOMM34']
# luad_genes_to_inspect_ims=['NFKBIA','S100A4','S100A13']
luad_genes_to_inspect_ims=['ETFB','CISD1']
l1k_2=l1k.copy()
for g in luad_genes_to_inspect_ims:
    model = linear_model.Lasso(alpha=0.02)
    model.fit(cp[cp_features],l1k[g])
    y_pred=model.predict(cp[cp_features])
    l1k_2['pred']=y_pred
    l1k_2['diff']=abs(l1k_2['pred']-l1k[g])
#     l1k_2[l1k_2['diff']<0.3]
    pert_to_vis_max=l1k_2[l1k_2['diff']<0.3].sort_values(by=['pred'],ascending=False)[:1].PERT.values[0]
    p,w=cp_data_repLevel[cp_data_repLevel['x_mutation_status']==pert_to_vis_max].sample(1)[['Metadata_Plate','Metadata_Well']].values[0]
    dict_imag_lookup[g]={}
    dict_imag_lookup[g]['high_'+pert_to_vis_max]=[p,w]
    pert_to_vis_min=l1k_2[l1k_2['diff']<0.3].sort_values(by=['pred'],ascending=False)[-1:].PERT.values[0]
    p,w=cp_data_repLevel[cp_data_repLevel['x_mutation_status']==pert_to_vis_min].sample(1)[['Metadata_Plate','Metadata_Well']].values[0]
    dict_imag_lookup[g]['low_'+pert_to_vis_min]=[p,w]
#     print(g,pert_to_vis_max,pert_to_vis_min)

/home/ubuntu/workspace_rosetta/workspace/software/2021_Haghighi_NeurIPS_Dataset_submitted/utils/readProfiles.py:42: DtypeWarning: Columns (1023,1028) have mixed types. Specify dtype option on import or set low_memory=False.
  l1k_data_repLevel=pd.read_csv(dataDir+'/L1000/replicate_level_l1k.csv.gz')


In [6]:
dict_imag_lookup

{'ETFB': {'high_BRAF_p.V600E': [52666, 'i23'], 'low_MYC_WT.c': [52653, 'k13']},
 'CISD1': {'high_AKT1_p.D323Y': [52649, 'j12'],
  'low_BRAF_p.G469S': [52652, 'i19']}}

## Run the following on DGX where luad compressed images currently reside

In [ ]:
where2move='/dgx1nas1/storage/data/marziehhaghighi/rosetta_luad_ims/'
luad_im_add='/dgx1nas1/storage/data/jarevalo/luad/dp_project/outputs/compressed/images/'
for g in dict_imag_lookup:
    print(dict_imag_lookup[g])
    geneName=g
    os.system('mkdir '+where2move+g)
    perts_dict=dict_imag_lookup[g].keys()
    for pe in perts_dict:
        os.system('mkdir '+where2move+g+'/'+pe)
        p,w=dict_imag_lookup[g][pe]
        print(p,w)
        plate_path=luad_im_add+str(p)
        list_plate_ims=os.listdir(plate_path)
        random_ims=[l for l in list_plate_ims if ('_'+w+'_' in l) and ('s5' in l)]
        print(random_ims)
#         fgsdfsfd
#         random_im=random.choice(random_ims)
        for r in random_ims:
            command='cp "'+plate_path+'/'+r+'" "'+where2move+g+'/'+pe+'/"'
            os.system(command)

## Check feature importance for a landmark gene of interest

In [ ]:
##############################
l1k,l1k_features_gn=rename_affyprobe_to_genename(l1k,l1k_features)

for g in ge_features_under_test:
#     viz = FeatureImportances(linear_model.Lasso(),relative=True, topn=20)
#     viz = FeatureImportances(linear_model.Lasso(alpha=0.001), labels=l1k_features_gn, relative=False,topn=20)
    print(g)
#     model = linear_model.LinearRegression()
    model = linear_model.Lasso(alpha=0.02)
    model.fit(cp[cp_features],l1k[g])
# #     model = RandomForestClassifier(n_estimators=10),colormap=single_blue_cmap
#     viz = FeatureImportances(model,topn=20,colormap='coolwarm')    
    viz = FeatureImportances(model,topn=20,colormap=single_blue_cmap)    
    
    viz.fit(cp[cp_features],l1k[g])
    viz.show()